## Description
---
This notebook is for comparing HYCOM surface temperatures with drifters.

Drifter data can be obtained at:
- https://www.aoml.noaa.gov/phod/gdp/data.php
- Generated URL for download (generated at bottom of download page): http://osmc.noaa.gov/erddap/tabledap/gdp_interpolated_drifter.csv?ID%2CWMO%2Cexpno%2Clongitude%2Clatitude%2Clon360%2Ctime%2Cdeploy_date%2Cdeploy_lat%2Cdeploy_lon%2Cend_date%2Cend_lat%2Cend_lon%2Cdrogue_lost_date%2Ctypedeath%2Ctypebuoy%2Ctemp%2Cve%2Cvn%2Cerr_temp%2Cerr_lat%2Cerr_lon%2CDeployingShip%2CDeploymentStatus%2CBuoyTypeManufacturer%2CBuoyTypeSensorArray%2CCurrentProgram%2CPurchaserFunding%2CSensorUpgrade%2CTransmissions%2CDeployingCountry%2CDeploymentComments%2CManufactureYear%2CManufactureMonth%2CManufactureSensorType%2CManufactureVoltage%2CFloatDiameter%2CSubsfcFloatPresence%2CDrogueType%2CDrogueLength%2CDrogueBallast%2CDragAreaAboveDrogue%2CDragAreaOfDrogue%2CDragAreaRatio%2CDrogueCenterDepth%2CDrogueDetectSensor&longitude%3E=0&longitude%3C=60&latitude%3E=-50&latitude%3C=-10&time%3E=2009-01-01T00%3A00%3A00Z%09&time%3C=2012-03-31T23%3A59%3A59Z&orderBy(%22time%22)

http://osmc.noaa.gov/erddap/tabledap/gdp_interpolated_drifter.csv?ID%2CWMO%2Cexpno%2Clongitude%2Clatitude%2Clon360%2Ctime%2Cdeploy_date%2Cdeploy_lat%2Cdeploy_lon%2Cend_date%2Cend_lat%2Cend_lon%2Cdrogue_lost_date%2Ctypedeath%2Ctypebuoy%2Ctemp%2Cve%2Cvn%2Cerr_temp%2Cerr_lat%2Cerr_lon%2CDeployingShip%2CDeploymentStatus%2CBuoyTypeManufacturer%2CBuoyTypeSensorArray%2CCurrentProgram%2CPurchaserFunding%2CSensorUpgrade%2CTransmissions%2CDeployingCountry%2CDeploymentComments%2CManufactureYear%2CManufactureMonth%2CManufactureSensorType%2CManufactureVoltage%2CFloatDiameter%2CSubsfcFloatPresence%2CDrogueType%2CDrogueLength%2CDrogueBallast%2CDragAreaAboveDrogue%2CDragAreaOfDrogue%2CDragAreaRatio%2CDrogueCenterDepth%2CDrogueDetectSensor&longitude%3E=0&longitude%3C=60&latitude%3E=-10&latitude%3C=-50&time%3E=2009-01-01T00%3A00%3A00Z%09&time%3C=2012-03-31T23%3A59%3A59Z&orderBy(%22time%22)

HYCOM data for comparing to drifters is obtained by:
1. hyc2proj ?

In [1]:
# import glob
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.wkt import loads
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from scipy import stats, spatial
import cmocean.cm as cmo
import cmcrameri.cm as cmc
import seawater
import gsw

In [2]:
plt.rcParams['figure.facecolor']='white'
plt.rcParams['axes.facecolor']='white'

In [54]:
drifters = pd.read_csv('../Data/drifter/gdp_interpolated_drifter_4925_427d_94d7.csv',
                       usecols = ['ID', 'time (UTC)', 'longitude (degrees_east)',
                                  'latitude (degrees_north)', 'temp (degree_C)',
                                  'err_temp (degree_C)'], parse_dates = ['time (UTC)'])

drifters = drifters.rename(columns={'time (UTC)': 'time',
                                    'longitude (degrees_east)': 'longitude',
                                    'latitude (degrees_north)': 'latitude',
                                    'temp (degree_C)': 'temp',
                                    'err_temp (degree_C)': 'err_temp'})
drifters

,ID,longitude,latitude,time,temp,err_temp
0,41534,52.311,-39.122,2009-01-01 00:00:00+00:00,14.042,0.042039
1,44647,39.393,-23.831,2009-01-01 00:00:00+00:00,27.543,0.044617
2,54287,10.390,-29.191,2009-01-01 00:00:00+00:00,20.760,0.036442
3,54434,38.112,-37.947,2009-01-01 00:00:00+00:00,18.284,0.044499
4,59346,1.388,-33.246,2009-01-01 00:00:00+00:00,19.595,0.035592
...,...,...,...,...,...,...
334785,92975,22.304,-44.270,2012-03-31 18:00:00+00:00,10.372,0.034828
334786,92986,14.316,-41.560,2012-03-31 18:00:00+00:00,15.489,0.057974
334787,92989,9.063,-43.516,2012-03-31 18:00:00+00:00,9.914,0.037446
334788,98866,4.659,-11.805,2012-03-31 18:00:00+00:00,26.483,0.035935


In [4]:
model = xr.open_dataset('../Data/hycom_free_sst.nc')
model

<xarray.Dataset>
Dimensions:      (depth: 1, latitude: 444, longitude: 666, time: 1982)
Coordinates:
  * time         (time) datetime64[ns] 2009-01-01 2009-01-02 ... 2014-06-05
  * longitude    (longitude) float32 0.0 0.09022556 ... 59.909775 60.0
  * latitude     (latitude) float32 -50.0 -49.93051 ... -10.10637 -10.0
  * depth        (depth) float32 0.0
Data variables:
    mercator     int32 ...
    x            (longitude) float32 ...
    y            (latitude) float32 ...
    model_depth  (latitude, longitude) float32 ...
    temperature  (time, depth, latitude, longitude) float32 ...
Attributes:
    title:                     Pilot MyOcean reanalysis by TOPAZ4 (2003-2008)
    institution:               NERSC, Thormoehlens gate 47, N-5006 Bergen, No...
    history:                   Tue Mar 24 14:42:10 2020: ncrcat AGUDAILY_star...
    source:                    NERSC-HYCOM model fields
    references:                http://topaz.nersc.no
    field_type:                Files based on file type nersc_daily
    Conventions:               CF-1.4
    field_date:                2009-01-01
    bulletin_date:             2007-11-14
    NCO:                       4.6.6
    nco_openmp_thread_number:  1

In [20]:
test = drifters.groupby('ID')
test.groups

{34134: Int64Index([ 22268,  22326,  22384,  22442,  22500,  22558,  22616,  22674,
              22731,  22788,
             ...
             118917, 119030, 119141, 119252, 119363, 119474, 119585, 119696,
             119807, 119918],
            dtype='int64', length=1418),
 34164: Int64Index([122870, 122978, 123086, 123195, 123304, 123413, 123522, 123631,
             123740, 123849,
             ...
             237551, 237628, 237705, 237783, 237861, 237939, 238017, 238095,
             238173, 238251],
            dtype='int64', length=1283),
 34166: Int64Index([53321, 53384, 53447, 53510, 53573, 53636, 53700, 53764, 53828,
             53892,
             ...
             59337, 59404, 59471, 59538, 59605, 59672, 59739, 59806, 59873,
             59939],
            dtype='int64', length=106),
 34173: Int64Index([51276, 51340, 51404, 51468, 51532, 51596, 51660, 51724, 51788,
             51852, 51916, 51980, 52045, 52110, 52175, 52240, 52305, 52368,
             52431, 52494, 5

Possibly, the best way to get mean position of drifter to provide location of the mean temp, is by using the package *nvector*: https://github.com/pbrod/nvector

In [63]:
def NearestNeighbourDiff(grid, points, gridInf=1, pointsInf=1):
    ''' The KDTree method used below is outlined at the link below:
       https://github.com/blaylockbk/pyBKB_v3/blob/master/demo/KDTree_nearest_neighbor.ipynb
    with some minor modifications in order to produce the differences.
    '''
    
    # Try find lon and lat variables for gridded dataset
    try:
        gridLons = grid['longitude']
    except KeyError:
        try:
            gridLons  = grid['lon']
        except KeyError:
            print("Can't detect gridded dataset longitude dimension.")
            
    try:
        gridLats = grid['latitude']
    except KeyError:
        try:
            gridLats  = grid['lat']
        except KeyError:
            print("Can't detect gridded dataset latitude dimension.")
    
    # Try find lon and lat variables for point observations dataset
    try:
        pointsLons = points['longitude']
    except KeyError:
        try:
            pointsLons  = points['lon']
        except KeyError:
            print("Can't detect point observations longitude dimension.")
            
    try:
        pointsLats = points['latitude']
    except KeyError:
        try:
            pointsLats  = points['lat']
        except KeyError:
            print("Can't detect point observations latitude dimension.")
    
    # Create meshgrid for use with KDTree
    x,y = np.meshgrid(*(gridLons, gridLats))
    
    # Build KDTree from L4 grid
    print('Now building KDTree...')
    tree = spatial.cKDTree(np.column_stack([x.ravel(), y.ravel()]))
    
    # Gather obs coordinates into a numpy column stack
    obs_points = np.column_stack([pointsLons, pointsLats])
    
    # Obtain distances between L2 obs and nearest L4 cell and the index of the L4 cell as
    #   recorded in the KDTree object (tree). Setting upper bound limits distances between
    #   L2 obs and L4 cells and will speed up the operation.
    print('Now querying KDTree for point observations...')
    dist, idx = tree.query(obs_points, distance_upper_bound=0.05, n_jobs=-1)
    print('Tree query completed.')
    
    # Squeeze L4 SSTs and then stack to obtain vector of temperatures to match shape of
    #   L2 observations for easy differencing
    grid_VecData = grid['temperature'].squeeze().stack(vec=['longitude', 'latitude'])
    
    # Obtain the difference between the L2 obs and the nearest L4 cell
    gridPoint_diff = points['temp'] - grid_VecData[idx-1].data
    
    return gridPoint_diff, dist, idx

In [101]:
modelDrifter_Diff, dist, idx = NearestNeighbourDiff(model.isel(time=0), drifters)

Now building KDTree...
Now querying KDTree for point observations...
Tree query completed.


### Selecting drifter data matching date index in model

In [ ]:
drifters_sub = drifters.loc[(drifters['time'].dt.date == pd.Timestamp(np.datetime64(model.isel(time=0)['time'].values)))]